<a href="https://colab.research.google.com/github/jakobatgithub/unreverb/blob/main/ClassificationOfReverb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow_io

import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# load data
# paths of the tensorflow datasets
path_1 = '/content/drive/My Drive/dsr_project/data/but-czas_v1.0/wavs/'
path_2 = '/content/drive/My Drive/dsr_project/data/but-czas_v1.0/wavs/'
path_3 = '/content/drive/My Drive/dsr_project/data/HarvardWordList/'
path_4 = '/content/drive/My Drive/dsr_project/data/Anechoic/'

# load the tensorflow datasets
dataset_1 = tf.data.experimental.load(path_1 + "M_tf_randomIRFs_dataset")
dataset_2 = tf.data.experimental.load(path_2 + "F_tf_randomIRFs_dataset")
dataset_3 = tf.data.experimental.load(path_3 + "tf_randomIRFs_dataset")
dataset_4 = tf.data.experimental.load(path_4 + "tf_randomIRFs_dataset")
dataset = dataset_1.concatenate(dataset_2)
dataset = dataset.concatenate(dataset_3)
dataset = dataset.concatenate(dataset_4)

# determine size of the dataset
dataset_size = sum(1 for _ in dataset)
print(f"dataset_size: {dataset_size}")

def lambda_1(features, labels, targets):
  return (features, labels)

def lambda_2(features, labels, targets):
  return (features, targets)

def lambda_3(features, labels, targets):
  return labels

# obtain the datasets containing only labels or targets
label_dataset = dataset.map(lambda features, labels, targets: lambda_1(features, labels, targets))
target_dataset = dataset.map(lambda features, labels, targets: lambda_2(features, labels, targets))

# obtain all labels
all_labels = dataset.map(lambda features, labels, targets: lambda_3(features, labels, targets))

# transform all_labels dataset to numpy array
all_labels_np = []
for label in all_labels:
  all_labels_np.append(tfds.as_numpy(label))

all_labels_np = np.array(all_labels_np)

# determine unique labels
unique_labels, counts = np.unique(all_labels_np, return_counts=True)
print(f"unique_labels: {unique_labels}")
#print(counts)

# determine number of unique labels
nr_of_unique_labels = unique_labels.shape[0]
print(f"nr_of_unique_labels: {nr_of_unique_labels}")

# shuffle the dataset before splitting in train and validate!
label_dataset = label_dataset.shuffle(dataset_size)

# split dataset in train and validate
train_fraction = 0.8
validate_dataset_size = int(dataset_size * (1.0-train_fraction)) # 20 percent of dataset_size
train_dataset = label_dataset.skip(validate_dataset_size)
validate_dataset = label_dataset.take(validate_dataset_size)

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import Audio, display
import librosa
import numpy as np

for sample in train_dataset.shuffle(dataset_size).take(1):
  audio = sample[0][0].numpy().astype("float32")
  label = sample[1].numpy()

  # print(label)
  # break

  plt.plot(audio)
  plt.title(f"Label: {label}")
  plt.show()
  plt.close()

  display(Audio(audio, rate=8000))

  mel = librosa.feature.melspectrogram(
      y=audio, n_mels=128, hop_length=64, sr=8000, fmax=2000
  )

  mel /= np.max(mel)
  plt.imshow(mel[::-1, :], cmap="inferno")

In [ ]:
from scipy.signal.spectral import spectrogram
import tensorflow as tf
import tensorflow_io as tfio
from keras.preprocessing.sequence import pad_sequences

def pad_second_dim(input, desired_size):
    padding = tf.tile([[0]], tf.stack([tf.shape(input)[0], desired_size - tf.shape(input)[1]], 0))
    return tf.concat([input, padding], 1)

def preprocess(sample, target):
  audio = sample[0]
  label = target
  spectrogram = tfio.audio.spectrogram(audio, nfft=1024, window=1024, stride=64)
  spectrogram = tfio.audio.melscale(spectrogram, rate=8000, mels=128, fmin=0, fmax=2000)
  spectrogram /= tf.math.reduce_max(spectrogram)
  spectrogram = tf.expand_dims(spectrogram, axis=-1)

  spectrogram = tf.image.resize(spectrogram, (128, 128))
  spectrogram = tf.transpose(spectrogram, perm=(1, 0, 2))
  spectrogram = spectrogram[::-1, :, :]

  return spectrogram, label


dataset_check = train_dataset.map(lambda sample, target: preprocess(sample, target))

for x, y in dataset_check.shuffle(dataset_size).take(4):
  plt.imshow(x[:,:,0], cmap="inferno")
  plt.title(f"{x.shape}, label: {y.numpy()}")
  plt.show()
  plt.close()

In [ ]:
dataset_train = train_dataset.map(lambda sample, target: preprocess(sample, target))
dataset_train = dataset_train.cache()
dataset_train = dataset_train.shuffle(dataset_size)
dataset_train = dataset_train.batch(32)

dataset_validate = validate_dataset.map(lambda sample, target: preprocess(sample, target))
dataset_validate = dataset_validate.cache()
dataset_validate = dataset_validate.batch(32)

In [ ]:
from tensorflow.keras import models, layers

conv1d_filters = 16
conv1d_strides = 3
my_input_shape = (128, 128, 1)

model = models.Sequential()
model.add(layers.Conv2D(4, (3, 3), padding="same", activation="relu", input_shape=my_input_shape))
model.add(layers.MaxPool2D((2, 2)))

model.add(layers.Conv2D(8, (3, 3), padding="same", activation="relu", input_shape=my_input_shape))
model.add(layers.MaxPool2D((2, 2)))

model.add(layers.Conv2D(16, (3, 3), padding="same", activation="relu", input_shape=my_input_shape))
model.add(layers.MaxPool2D((2, 2)))

model.add(layers.Conv2D(32, (3, 3), padding="same", activation="relu", input_shape=my_input_shape))
model.add(layers.MaxPool2D((2, 2)))


model.add(layers.Flatten())
model.add(layers.Dense(nr_of_unique_labels, activation="softmax"))
model.summary()

model.compile(
    optimizer="adam",
    # loss=tf.keras.losses.MeanSquaredError(),
    # loss="binary_crossentropy",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

history = model.fit(
    dataset_train,
    epochs=300,
    validation_data=dataset_validate
)

In [ ]:
def render_history(history):
    plt.plot(history["loss"], label="loss")
    plt.plot(history["val_loss"], label="val_loss")
    plt.legend()
    plt.title("Our losses")
    plt.show()
    plt.close()

    plt.plot(history["accuracy"], label="accuracy")
    plt.plot(history["val_accuracy"], label="val_accuracy")
    plt.legend()
    plt.title("Our accuracies")
    plt.show()
    plt.close()


def compare_histories():
    for training_name, history in history_list.items():
        plt.plot(history["val_accuracy"], label=training_name)
    plt.legend()
    plt.title("Comparision of val_accuracy")
    plt.show()
    plt.close()

render_history(history.history)